# <font color='blue'> <center> SARSA </center> </font>

<div style="text-align:center">
    In this notebook we are going to implement a method that learns from experience and uses bootstrapping.
    It is known as SARSA because of the elements involved in the update rule:
</div>

\begin{equation}
\text{State}_t, \text{Action}_t, \text{Reward}_t, \text{State}_{t+1}, \text{Action}_{t+1}
\end{equation}

<br>

<div style="text-align:center">
    This method follows an on-policy strategy, in which the same policy that is optimized is responsible for scanning the environment.
</div>

## <font color='#2874A6'> Tabla de Contenido </font>

1. [Modules](#1)
2. [Setup Code](#2)
3. [Algorithm](#3)

<a name="1"></a>
## <font color='#0E6655'> 1. Modules </font> 